In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U --q transformers datasets accelerate sentencepiece wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
%cd '/content/drive/MyDrive/NLU_NCKH/notebook/res_data_preprocesed/'

/content/drive/.shortcut-targets-by-id/1btr8FZI1SJ1bZdaS-BqFfYlWxPaXfQOB/NLU_NCKH/notebook/res_data_preprocesed


# Import library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch
import datasets
from sklearn.metrics import *

# Load data

In [ ]:
df_train = pd.read_csv("train_res_preprocesed.csv")
df_dev = pd.read_csv("dev_res_preprocesed.csv")
df_test = pd.read_csv("test_res_preprocesed.csv")

print("Train: ", df_train.shape)
print("Dev: ",  df_dev.shape)
print("Test: ", df_test.shape )

Train:  (7028, 13)
Dev:  (771, 13)
Test:  (1938, 13)


# Prepare data for model `vinai/phobert-base`

In [ ]:
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, df, aspects, tokenizer, MAX_LEN, type_model="train") -> None:
    self.df = df
    self.aspects = aspects
    self.tokenizer = tokenizer
    self.MAX_LEN = MAX_LEN
    self.type_model = type_model
  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    review = row['review']
    aspects = row[self.aspects]

    inputs = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.MAX_LEN,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]

    if self.type_model == "test":
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

    label = self.label_one_hot(aspects)
    label = label.reshape(-1)
    return {
        'input_ids': torch.tensor(ids, dtype=torch.long),
        'attention_mask': torch.tensor(mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        'labels': torch.tensor(label, dtype=torch.float)
    }


  def label_one_hot(self, label):
    encoder = OneHotEncoder(categories=[[0, 1, 2, 3]], dtype='uint8', sparse=False)
    one_hot_outputs = encoder.fit_transform(label.values.reshape(-1, 1))
    return one_hot_outputs


MODEL_NAME = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Hyperparameters
MAX_LEN = 140
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 12
LEARNING_RATE = 1e-05

aspects = df_train.drop(columns='review').columns.tolist()

tokenizer_train = CustomDataset(df_train, aspects, tokenizer, MAX_LEN)
tokenizer_test = CustomDataset(df_test, aspects, tokenizer, MAX_LEN, "test")
tokenizer_dev = CustomDataset(df_dev, aspects, tokenizer, MAX_LEN)

print("Encoded: ",tokenizer_dev.__getitem__(1)['input_ids'].size())
print("Decoded: ",tokenizer.decode(tokenizer_dev.__getitem__(1)['input_ids'], skip_special_tokens=True))
print("Lable: ",tokenizer_dev.__getitem__(1)['labels'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Encoded:  torch.Size([256])
Decoded:  nay đi uống mới biết giá_thành hơi cao nhưng thật_sự đi_đôi với chất_lượng
Lable:  tensor([1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])


# Prepare DataLoader

In [ ]:
from torch.utils.data import DataLoader

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(tokenizer_train, **train_params)
validating_loader = DataLoader(tokenizer_dev, **test_params)
testing_loader = DataLoader(tokenizer_test, **test_params)

for k, v in next(iter(training_loader)).items():
  print(f"{k}: {v.size()}")

input_ids: torch.Size([32, 127])
attention_mask: torch.Size([32, 127])
token_type_ids: torch.Size([32, 127])
labels: torch.Size([32, 48])


# Traing loop

## Define model for training loop

In [ ]:
import torch
from torch import nn

class phoBERTClass(torch.nn.Module):
    def __init__(self, MODEL_NAME, num_labels):
        super(phoBERTClass, self).__init__()
        self.pretrained_bert = AutoModel.from_pretrained(MODEL_NAME, output_hidden_states=True)
        self.dropout = nn.Dropout(0.2)
        self.num_labels = num_labels
        self.classifier = nn.ModuleList(
            [nn.Linear(self.pretrained_bert.config.hidden_size * 4, 4)
              for _ in range(self.num_labels)
            ]
          )

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.pretrained_bert(input_ids,
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                      )
        hidden_states = outputs.hidden_states

        # Concatenate last 4 hidden states
        pooled_output = torch.cat(tuple([hidden_states[i] for i in range(-4, 0)]), dim=-1)
        x = self.dropout(pooled_output[:, 0, :])

        outputs = [nn.functional.softmax(dense_layer(x), dim=-1) for dense_layer in self.classifier]

        return torch.cat(outputs, dim=-1)

## Define train and evaluate

In [ ]:
from tqdm.auto import tqdm
from sklearn import metrics
import time

def train_epoch(model, optimizer, loss_fn, train_loader, device, progress_bar, epoch):
    model.train()
    losses = []
    for index, data in tqdm(enumerate(train_loader), desc=f'Training {epoch}', total=len(train_loader)):
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          targets = data['labels'].to(device, dtype = torch.float)

          outputs = model(ids, mask, token_type_ids)

          optimizer.zero_grad()
          loss = loss_fn(outputs, targets)
          if index%100 == 0 or index == (len(training_loader) - 1):
              print(
                "| Epoch {:3d} | Index: {:4d} | Train Loss {:8.3f}"
                .format(
                    epoch, index, loss.item()
                  )
              )
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          progress_bar.update(1)

          losses.append(loss.item())

    return sum(losses)/len(losses)

def evaluate_epoch(model, loss_fn, valid_loader, device):
      model.eval()
      fin_targets=[]
      fin_outputs=[]
      losses = []
      with torch.no_grad():
          for _, data in tqdm(enumerate(valid_loader, 0), desc="Evaluating", total=len(valid_loader)):
              ids = data['input_ids'].to(device, dtype = torch.long)
              mask = data['attention_mask'].to(device, dtype = torch.long)
              token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
              targets = data['labels'].to(device, dtype = torch.float)

              outputs = model(ids, mask, token_type_ids)

              loss = loss_fn(outputs, targets)
              losses.append(loss)
              fin_targets.extend(targets.cpu().detach().numpy().tolist())
              fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      return {
          "loss": sum(losses)/len(losses),
        }

def train(model, model_name, save_model, optimizer, loss_fn, train_loader,
          valid_loader, num_epochs, device, num_training_steps):
      progress_bar = tqdm(range(num_training_steps), desc="Training")
      train_losses = []
      metrics_eval = []
      best_loss_eval = 100
      times = []

      for epoch in range(1, num_epochs+1):
          epoch_start_time = time.time()
          # Training
          train_loss = train_epoch(model, optimizer, loss_fn, train_loader, device, progress_bar, epoch)
          train_losses.append(train_loss)

          # Evaluation
          eval_metric = evaluate_epoch(model, loss_fn, valid_loader, device)
          metrics_eval.append(eval_metric)

          # Save best model
          if eval_metric['loss'] < best_loss_eval:
              torch.save(model.state_dict(), save_model + f'/{model_name}.pt')

          times.append(time.time() - epoch_start_time)
          # Print loss, acc end epoch
          print("-" * 80)
          print(
              "| End of epoch {:3d} | Time: {:5.2f}s | Train Loss {:8.3f} | Valid Loss {:5.3f}"
              .format(
                  epoch, time.time() - epoch_start_time, train_loss, eval_metric["loss"]
              )
          )
          print("-" * 80)

      # Load best model
      model.load_state_dict(torch.load(save_model + f'/{model_name}.pt'))
      model.eval()
      metrics = {
          'train_loss': train_losses,
          'metric_eval': metrics_eval,
          'time': times
      }
      return model, metrics

## Initialize training loop

In [ ]:
from transformers import get_scheduler

save_model = "/content/drive/MyDrive/NLU_NCKH/notebook/Checkpoints/Restaurant_BERT"
EPOCHS = 10
num_labels = len(df_train.columns[1:])
model = phoBERTClass(MODEL_NAME, num_labels)

def loss_fn(outputs, targets):
    return torch.nn.BCELoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

num_training_steps = EPOCHS * len(training_loader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
best_model, metrics = train(
    model, "Restaurant_BERT", save_model, optimizer, loss_fn, training_loader,
    validating_loader, EPOCHS, device, num_training_steps
)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

  0%|          | 0/2640 [00:00<?, ?it/s]

Epoch/index: 1 | 0, Loss:  0.7067446708679199
Epoch/index: 1 | 200, Loss:  0.2179999053478241
Evaluating
-----------------------------------------------------------
| End of epoch   1 | Time: 176.23s | Train Loss    0.302 | Valid Loss 0.198
-----------------------------------------------------------
Epoch/index: 2 | 0, Loss:  0.1966852843761444
Epoch/index: 2 | 200, Loss:  0.17983654141426086
Evaluating
-----------------------------------------------------------
| End of epoch   2 | Time: 154.76s | Train Loss    0.186 | Valid Loss 0.170
-----------------------------------------------------------
Epoch/index: 3 | 0, Loss:  0.16087549924850464
Epoch/index: 3 | 200, Loss:  0.14161565899848938
Evaluating
-----------------------------------------------------------
| End of epoch   3 | Time: 154.55s | Train Loss    0.156 | Valid Loss 0.137
-----------------------------------------------------------
Epoch/index: 4 | 0, Loss:  0.12717723846435547
Epoch/index: 4 | 200, Loss:  0.1312636733055114

# Inference

## Load model

In [ ]:
save_model = "/content/drive/MyDrive/NLU_NCKH/notebook/model/phobert-base-ABSA/"
model_name = "phoBert_base_ABSA"
device = "cuda" if torch.cuda.is_available() else "cpu"
num_labels = len(df_train.columns[1:])
model = phoBERTClass(MODEL_NAME, num_labels)

model.load_state_dict(torch.load(save_model + f"/{model_name}.pt"))
model.to(device)
model.eval()

phoBERTClass(
  (pretrained_bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm)

## Predict

In [ ]:
from tqdm import tqdm
def predict(model, testing_loader, device):
  predictions = []
  with torch.no_grad():
      for _, data in tqdm(enumerate(testing_loader, 0), desc="Predicting", total=len(testing_loader)):
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)

          outputs = model(ids, mask, token_type_ids)
          predictions.extend(outputs.reshape(-1, 12, 4).argmax(axis=-1).cpu().detach().numpy())

      return np.array(predictions)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def print_acsa_pred(replacements, categories, y_pred, y_true):
    sentiments = map(lambda x: replacements[x], y_pred)
    y_true = map(lambda x: replacements[x], y_true)
    predicted = []
    actual = []
    for category, sentiment, y in zip(categories, sentiments, y_true):
        if sentiment: predicted.append(f'{category}, {sentiment}')
        if y: actual.append(f'{category}, {y}')

    print("Predicted: ", ' | '.join(predicted))
    print("Ground Truth: ", ' | '.join(actual))

def label_one_hot(labels):
    len = labels.shape[0]
    labels = labels.to_numpy().reshape(-1, 1)
    encoder = OneHotEncoder(categories=[[0, 1, 2, 3]], dtype='uint8', sparse=False)
    return encoder.fit_transform(labels).reshape(len, -1, 4)

aspects = df_test.columns[1:]

y_true = df_test[aspects].values
y_pred = predict(best_model, testing_loader, device)

In [ ]:
replacements = {0: None, 1: 'positive', 2: 'negative', 3: 'neutral'}

for i in range(5):
  print("-"*100)
  print('Example:', df_test['review'][i])
  print_acsa_pred(replacements, aspects, y_pred[i], y_true[i])

----------------------------------------------------------------------------------------------------
Example: red_heart bánh được bao_gói bao_bì khá lạ là túi nilong được băng dập kín để hút khí chứ kfai túi giấy như các cửa_hàng khác flexed_biceps_light_skin_tone
Predicted:  RESTAURANT#MISCELLANEOUS, neutral
Actual:  RESTAURANT#MISCELLANEOUS, neutral
----------------------------------------------------------------------------------------------------
Example: star soda and lemon thì cũng tạm được
Predicted:  DRINKS#STYLE&OPTIONS, negative
Actual:  DRINKS#QUALITY, negative
----------------------------------------------------------------------------------------------------
Example: mình sẽ không bao_giờ đặt_hàng giao hàng thế này nữa làm_ăn cẩu_thả thua xa koi mình cũng đặt_hàng koi nhiều nhưng cbh thấy tình_trạng thế này
Predicted:  RESTAURANT#GENERAL, positive
Actual:  SERVICE#GENERAL, positive
--------------------------------------------------------------------------------------------

## Aspect Detection

In [ ]:
from sklearn.metrics import classification_report

aspect_test = []
aspect_pred = []
y_test_argmax = df_test[aspects].to_numpy()

for row_test, row_pred in zip(y_test_argmax, y_pred):
    for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
        aspect_test.append(bool(col_test) * aspects[index])
        aspect_pred.append(bool(col_pred) * aspects[index])

aspect_report = classification_report(aspect_test, aspect_pred, digits=4, zero_division=1, output_dict=True)
print(classification_report(aspect_test, aspect_pred, digits=4, zero_division=1))

                          precision    recall  f1-score   support

                             0.9743    0.9872    0.9807     20627
        AMBIENCE#GENERAL     0.9078    0.8238    0.8637       227
           DRINKS#PRICES     0.9487    0.7872    0.8605        47
          DRINKS#QUALITY     0.8680    0.8424    0.8550       203
    DRINKS#STYLE&OPTIONS     0.8351    0.6279    0.7168       129
             FOOD#PRICES     0.8598    0.8214    0.8402       112
            FOOD#QUALITY     0.9125    0.8664    0.8889       554
      FOOD#STYLE&OPTIONS     0.8561    0.7757    0.8139       437
        LOCATION#GENERAL     0.9663    0.8269    0.8912       104
      RESTAURANT#GENERAL     0.8408    0.6733    0.7478       251
RESTAURANT#MISCELLANEOUS     0.8036    0.6207    0.7004       145
       RESTAURANT#PRICES     0.8762    0.7863    0.8288       117
         SERVICE#GENERAL     0.9502    0.8812    0.9144       303

                accuracy                         0.9655     23256
        

## Polarity Detection

In [ ]:
y_test_flat = y_test_argmax.flatten()
y_pred_flat = y_pred.flatten()
target_names = list(map(str, replacements.values()))

polarity_report = classification_report(y_test_flat, y_pred_flat, digits=4, output_dict=True)
print(classification_report(y_test_flat, y_pred_flat, target_names=target_names, digits=4))

              precision    recall  f1-score   support

        None     0.9743    0.9872    0.9807     20627
    positive     0.6253    0.5333    0.5757       435
    negative     0.6319    0.4819    0.5468       691
     neutral     0.7805    0.7572    0.7687      1503

    accuracy                         0.9488     23256
   macro avg     0.7530    0.6899    0.7180     23256
weighted avg     0.9450    0.9488    0.9465     23256



## Aspect + Polarity

In [ ]:
aspect_polarity_test = []
aspect_polarity_pred = []

for row_test, row_pred in zip(y_test_argmax, y_pred):
    for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
        aspect_polarity_test.append(f'{aspects[index]},{replacements[col_test]}')
        aspect_polarity_pred.append(f'{aspects[index]},{replacements[col_pred]}')


aspect_polarity_report = classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1, output_dict=True)
print(classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1))

                                   precision    recall  f1-score   support

            AMBIENCE#GENERAL,None     0.9769    0.9889    0.9829      1711
        AMBIENCE#GENERAL,negative     0.5000    0.0667    0.1176        30
         AMBIENCE#GENERAL,neutral     0.8284    0.8750    0.8511       160
        AMBIENCE#GENERAL,positive     0.5758    0.5135    0.5429        37
               DRINKS#PRICES,None     0.9947    0.9989    0.9968      1891
           DRINKS#PRICES,negative     0.7105    0.8438    0.7714        32
            DRINKS#PRICES,neutral     1.0000    0.0000    0.0000        10
           DRINKS#PRICES,positive     1.0000    0.2000    0.3333         5
              DRINKS#QUALITY,None     0.9816    0.9850    0.9833      1735
          DRINKS#QUALITY,negative     0.7500    0.2500    0.3750        48
           DRINKS#QUALITY,neutral     0.7357    0.7984    0.7658       129
          DRINKS#QUALITY,positive     0.4146    0.6538    0.5075        26
        DRINKS#STYLE&OPT

# Summary

In [ ]:
# Create dict score
aspect_dict = aspect_report['macro avg']
aspect_dict['accuracy'] = aspect_report['accuracy']

polarity_dict  = polarity_report['macro avg']
polarity_dict['accuracy'] = polarity_report['accuracy']

aspect_polarity_dict = aspect_polarity_report['macro avg']
aspect_polarity_dict['accuracy'] = aspect_polarity_report['accuracy']

# Create dataframe for dict_score
df_report = pd.DataFrame.from_dict([aspect_dict, polarity_dict, aspect_polarity_dict])
df_report.index = ['Aspect Detection', 'Polarity Detection', 'Aspect + Polarity']
df_report.drop('support', axis=1)

,precision,recall,f1-score,accuracy
Aspect Detection,0.889406,0.781997,0.828492,0.965041
Polarity Detection,0.729676,0.669379,0.696015,0.945562
Aspect + Polarity,0.760695,0.559804,0.568096,0.945562
